# Load

In [1]:
import numpy as np
import scipy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import activations
from tensorflow.keras.backend import clear_session
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks

from math import ceil
import matplotlib.pyplot as plt

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
data = np.load("../output/data-10.npz", allow_pickle=True)
sorted(data.files)
X = data['X']
y = data['y']
data.close()

In [ ]:
#This code is used on tfidf data!
#X = X[()].toarray()
#input_dim = X.shape[1]

In [3]:
#should be (None, 40, 1)
X_train = X.reshape((X.shape[0],X.shape[1],1))

In [4]:
print(X.shape,
X_train.shape)

(52, 40) (52, 40, 1)


# Test Model

In [17]:
def create_model(lr, _size):
    clear_session()

    model = Sequential()
    model.add(SeparableConv1D(input_shape = (_size, 1), filters = 128, kernel_size = 5, padding = 'valid'))
    model.add(MaxPooling1D(2))
    model.add(LSTM(_size,return_sequences = True, implementation=2))
    model.add(Dropout(0.7))
    model.add(Activation('relu'))
    model.add(LSTM(256,return_sequences = True))
    model.add(Activation('relu'))
    model.add(LSTM(128,return_sequences = True))
    model.add(Activation('relu'))
    model.add(LSTM(64,return_sequences = True))
    model.add(Activation('relu'))
    model.add(LSTM(32,return_sequences = True))
    model.add(Activation('relu'))
    model.add(LSTM(16))
    model.add(Activation('relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizers.RMSprop(learning_rate=lr),
        metrics=['acc'])

    return model

model = create_model(lr=0.001, _size=X.shape[1])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv1d (SeparableC (None, 38, 128)           259       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 19, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 19, 40)            27040     
_________________________________________________________________
dropout (Dropout)            (None, 19, 40)            0         
_________________________________________________________________
activation (Activation)      (None, 19, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 19, 256)           304128    
_________________________________________________________________
activation_1 (Activation)    (None, 19, 256)           0

In [4]:
def create_model(lr):
    clear_session()

    model = Sequential()

    model.add(SeparableConv1D(input_shape = (40, 1), filters = 256, kernel_size = 3, padding = 'valid'))
    model.add(Activation(activations.relu))
    model.add(SeparableConv1D(filters = 256, kernel_size = 3, padding = 'valid'))
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size = 2, padding = 'valid'))

    model.add(SeparableConv1D(filters = 256, kernel_size = 3, padding = 'valid'))
    model.add(Activation(activations.relu))
    model.add(SeparableConv1D(filters = 256, kernel_size = 3, padding = 'valid'))
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size = 2, padding = 'valid'))
    model.add(Dropout(rate=0.5))
    
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation(activations.relu))
    model.add(Dropout(rate=0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizers.RMSprop(learning_rate=lr),
        metrics=['acc'])

    return model

model = create_model(lr=0.001)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv1d (SeparableC (None, 38, 256)           515       
_________________________________________________________________
activation (Activation)      (None, 38, 256)           0         
_________________________________________________________________
separable_conv1d_1 (Separabl (None, 36, 256)           66560     
_________________________________________________________________
activation_1 (Activation)    (None, 36, 256)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 18, 256)           0         
_________________________________________________________________
separable_conv1d_2 (Separabl (None, 16, 256)           66560     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 256)           0

In [25]:
callback = callbacks.EarlyStopping(min_delta = 0.02, monitor='acc', patience=20, restore_best_weights=True)

model = create_model(lr=0.001, _size=X.shape[1])

history = model.fit(X_train,
                   y,
                   #validation_split=0.2,
                   shuffle=True,
                   epochs=50,
                   batch_size=ceil(X.shape[0]/20),
                   #callbacks=[callback],
                   #class_weight={0: compute_class_weight('balanced', np.unique(y), y)[0], 1: compute_class_weight('balanced', np.unique(y), y)[1]}#{0: 1.0, 1: 30.0}
                    )

Epoch 1/50
18/18 [==============================] - 13s 44ms/step - loss: 0.6918 - acc: 0.5577
Epoch 2/50
18/18 [==============================] - 1s 46ms/step - loss: 0.6660 - acc: 0.6538
Epoch 3/50
18/18 [==============================] - 1s 46ms/step - loss: 0.6520 - acc: 0.6538
Epoch 4/50
18/18 [==============================] - 1s 45ms/step - loss: 0.6510 - acc: 0.6538
Epoch 5/50
18/18 [==============================] - 1s 46ms/step - loss: 0.6106 - acc: 0.6923
Epoch 6/50
18/18 [==============================] - 1s 46ms/step - loss: 0.5506 - acc: 0.8077
Epoch 7/50
18/18 [==============================] - 1s 44ms/step - loss: 0.5541 - acc: 0.7692
Epoch 8/50
18/18 [==============================] - 1s 48ms/step - loss: 0.5375 - acc: 0.7885
Epoch 9/50
18/18 [==============================] - 1s 45ms/step - loss: 0.5043 - acc: 0.8077
Epoch 10/50
18/18 [==============================] - 1s 45ms/step - loss: 0.4948 - acc: 0.8077
Epoch 11/50
18/18 [==============================] - 1s 45

# Plot stuff

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Grid Optimizer

In [ ]:
#wrapping model
gridmodel = KerasClassifier(build_fn=create_model, input_dim=input_dim, class_weight={0: 1.0, 1: 30.0})

_batch_size      = [20]
_optimizer       = ['RMSprop']
_dense_width     = [64, 128]
_dropout_value   = [0.5]
_n_hidden_layers = [2, 3]

param_grid = dict(
    batch_size      =_batch_size, 
    optimizer       =_optimizer, 
    dense_width     =_dense_width,
    dropout_value   =_dropout_value,
    n_hidden_layers =_n_hidden_layers
    )

grid = GridSearchCV(estimator=gridmodel, param_grid=param_grid, scoring='accuracy', n_jobs=16, cv=6, verbose=2)

In [ ]:
callback = callbacks.EarlyStopping(monitor='val_acc', patience=5)

grid_result = grid.fit(X, 
                       y, 
                       epochs = 100,
                       validation_split=0.2,
                       callbacks=[callback])

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))